In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_memory="6000m"
launcher.conf.set("spark.sql.warehouse.dir","hdfs://bd-hm:9000/user/hive/warehouse")
launcher.conf.set("spark.sql.catalogImplementation","hive")

In [2]:
sql("create database if not exists common")
sql("use common")

Intitializing Scala interpreter ...

Spark Web UI available at http://bd-hm:8088/proxy/application_1573537526242_0001
SparkContext available as 'sc' (version = 2.4.4, master = yarn, app id = application_1573537526242_0001)
SparkSession available as 'spark'


res0: org.apache.spark.sql.DataFrame = []


In [3]:
val ratings =sc.textFile("/hadoop-user/data/ratings").map{line=>val splits=line.split("\\s+");(splits(0),splits(1))}.flatMapValues(movie_id=>movie_id.split("#")).toDF("user_id","movie_id").persist
//ratings.take(3).foreach(println)
ratings.createOrReplaceTempView("ratings")
                                                               
val movies=sc.textFile("/hadoop-user/data/movies").map{line=>val splits=line.split("#");(splits(0),splits(1))}.toDF("movie_id","movie_name").persist
//movies.take(3).foreach(println)
movies.createOrReplaceTempView("movies")
                                                                                                     
                                                                                                     

//orders.write.parquet("orders.parquet")

//spark.read.parquet("orders.parquet")
//ratingsnew.createOrReplaceTempView("ratingsnew")
                                                               

ratings: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: string, movie_id: string]
movies: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [movie_id: string, movie_name: string]


In [4]:
ratings.show(10)

+-------+--------+
|user_id|movie_id|
+-------+--------+
|      1|       1|
|      1|    1097|
|      1|    1907|
|      1|    2321|
|      1|    2018|
|      1|     260|
|      1|     938|
|      1|    1246|
|      1|    2028|
|      1|     150|
+-------+--------+
only showing top 10 rows



In [5]:
movies.show(10)

+--------+--------------------+
|movie_id|          movie_name|
+--------+--------------------+
|       1|    Toy Story (1995)|
|       2|      Jumanji (1995)|
|       3|Grumpier Old Men ...|
|       4|Waiting to Exhale...|
|       5|Father of the Bri...|
|       6|         Heat (1995)|
|       7|      Sabrina (1995)|
|       8| Tom and Huck (1995)|
|       9| Sudden Death (1995)|
|      10|    GoldenEye (1995)|
+--------+--------------------+
only showing top 10 rows



In [7]:
sql("""select r1.user_id as user_id1,r2.user_id as user_id2,count(r1.movie_id)as cnt,collect_set(m.movie_name)
from ratings r1, ratings r2,movies m
where r1.movie_id=r2.movie_id AND r2.movie_id=m.movie_id
group by r1.user_id,r2.user_id
having cnt>50
order by cnt 

""").show(10)

+--------+--------+---+-----------------------+
|user_id1|user_id2|cnt|collect_set(movie_name)|
+--------+--------+---+-----------------------+
|    1077|    2180| 51|   [Fantasia (1940),...|
|     118|    1866| 51|   [E.T. the Extra-T...|
|    1019|    2921| 51|   [Planet of the Ap...|
|    1053|     306| 51|   [Boiler Room (200...|
|      10|     956| 51|   [Fantasia (1940),...|
|    1016|    2771| 51|   [Entrapment (1999...|
|    1094|     955| 51|   [Indiana Jones an...|
|    1117|     369| 51|   [Dave (1993), Men...|
|    1125|    1209| 51|   [Primal Fear (199...|
|    1128|     843| 51|   [High Fidelity (2...|
+--------+--------+---+-----------------------+
only showing top 10 rows

